# Creating a basic chat experience with context variables

In this example, we show how you can build a simple chat bot by sending and updating context with your requests. 

We introduce the Context Variables object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The context is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the context on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the context gets populated with the history of the conversation. During each new run of the kernel, the context can provide the AI with its variables' content. 

In [ ]:
# !python -m pip install semantic-kernel==0.4.3.dev0

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-3.5", OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id))


Let's define a prompt outlining a dialogue chat bot.

In [3]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function

In [4]:
chat_function = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.7,
    top_p=0.5)

Initialize your context

In [5]:
context = kernel.create_new_context()
context["history"] = ""

In [20]:


def print_formatted_json(context):
    import json
    formatted_json = json.loads(context.model_dump_json())
    print(json.dumps(formatted_json, indent=4))



In [21]:
print_formatted_json(context=context)

{
    "memory": {},
    "variables": {
        "variables": {
            "input": "",
            "history": ""
        }
    },
    "skill_collection": {
        "data": {
            "_global_functions_": {
                "chatbot": {}
            }
        }
    }
}


Chat with the Bot

In [22]:
context["user_input"] = "Hi, I'm looking for book suggestions"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)
print_formatted_json(context=context)

Variable `$history` not found


Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?
{
    "memory": {},
    "variables": {
        "variables": {
            "input": "Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?",
            "history": "",
            "user_input": "Hi, I'm looking for book suggestions"
        }
    },
    "skill_collection": {
        "data": {
            "_global_functions_": {
                "chatbot": {}
            }
        }
    }
}


Update the history with the output

In [23]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?



Keep Chatting!

In [24]:
print_formatted_json(context=context)

{
    "memory": {},
    "variables": {
        "variables": {
            "input": "Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?",
            "history": "\nUser: Hi, I'm looking for book suggestions\nChatBot: Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?\n",
            "user_input": "Hi, I'm looking for book suggestions"
        }
    },
    "skill_collection": {
        "data": {
            "_global_functions_": {
                "chatbot": {}
            }
        }
    }
}


In [3]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [26]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: I would recommend "The Histories" by Herodotus. It is considered one of the earliest works of history and provides a detailed account of ancient Greece, including the Persian Wars. Another great book is "The Republic" by Plato, which explores various philosophical ideas and includes discussions on justice, politics, and the ideal society.


In [27]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: "The Histories" by Herodotus is a historical account of the Persian Wars and the events leading up to it. It covers various aspects of ancient Greek society, including their customs, traditions, and political systems. It also includes stories and anecdotes about famous figures such as King Leonidas of Sparta and the Battle of Thermopylae. 

"The Republic" by Plato is a philosophical work that explores the concept of justice and the ideal society. It presents a dialogue between Socrates and various other characters, discussing topics such as the nature of justice, the role of the philosopher, and the different forms of government. It also delves into the concept of the "philosopher king" and the importance of education in shaping a just society.


In [28]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: If you read "The Histories" by Herodotus, you will learn about various aspects of Greek history, particularly the Persian Wars. You will gain insights into the political systems, customs, and traditions of ancient Greece. You will also learn about significant events such as the Battle of Marathon, the Battle of Thermopylae, and the Battle of Salamis. Additionally, you will learn about the interactions between the Greeks and the Persians, as well as the cultural and societal differences between the two civilizations.

It's important to note that while "The Histories" provides valuable historical information, it is also known for including some mythical and exaggerated elements. However, it still remains a significant source for understanding ancient Greek history.


In [29]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Certainly! Here are some more books you could consider reading about Greek history:

1. "The Peloponnesian War" by Thucydides - This book provides a detailed account of the Peloponnesian War between Athens and Sparta, offering insights into the political and military strategies of the time.

2. "The Greek Way" by Edith Hamilton - This book explores the cultural and intellectual achievements of ancient Greece, including their philosophy, literature, and art.

3. "The Greek World 479-323 BC" by Simon Hornblower - This book provides a comprehensive overview of the political, social, and cultural developments in Greece during the Classical period.

4. "The Rise and Fall of Athens: Nine Greek Lives" by Plutarch - This book offers biographies of nine influential figures in ancient Greece, including Pericles, Themistocles, and Alcibiades, providing insights into the political and military history of Athens.

5. "Anci

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [30]:
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: I would recommend "The Histories" by Herodotus. It is considered one of the earliest works of history and provides a detailed account of ancient Greece, including the Persian Wars. Another great book is "The Republic" by Plato, which explores various philosophical ideas and includes discussions on justice, politics, and the ideal society.

User: that sounds interesting, what is it about?
ChatBot: "The Histories" by Herodotus is a historical account of the Persian Wars and the events leading up to it. It covers various aspects of ancient Greek society, including their customs, traditions, and political systems. It also includes stories and anecdotes about famous figures such as King Leonidas of Sparta and the Battle of Thermop

In [7]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from semantic_kernel.orchestration.sk_context import SKContext

kernel = sk.Kernel()

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(
        deployment_name=deployment,
        endpoint=endpoint,
        api_key=api_key
        )
    )

sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

chat_function = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.7,
    top_p=0.5)


async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"


context = kernel.create_new_context()
context["history"] = ""
while(True):
    input_text = input("User: ")
    await chat(input_text)
    if input_text == "bye":
        break


Variable `$history` not found


User: Hello
ChatBot: Hi there! How can I assist you today?
User: What is fun today? 
ChatBot: There are many fun things you can do today! You could go for a walk in the park, watch a movie, read a book, try a new recipe, or even learn a new hobby. It all depends on what you enjoy doing!
User: Bye
ChatBot: Goodbye! Have a great day!
User: bye
ChatBot: Goodbye! Take care!
